In [1]:
import os
from pathlib import Path
import shutil

In [2]:
#SELECT THE SIZE OF THE NEW SF THAT YOU WANT TO CREATE
SF=5

In [3]:
#SET VARIABLES
PATH=os.getcwd().replace("\\", "/")+'/dags/data/sf_'+str(SF)
print(PATH)

/Users/alfio/projects/tpcdi-benchmark-postgres/src/dags/data/sf_5


In [4]:
p = Path('.')
dags_templates_path = p / 'dags_temp'
dags_sf_path = p / 'dags' / f'sf_{SF}'

shutil.copytree(dags_templates_path , dags_sf_path)
with open(dags_sf_path / 'full_historical_load_etl_dag.py', "r") as file:
    content = file.read()
modified_content = content.replace('INPUT_SF', str(SF))
with open(dags_sf_path / 'full_historical_load_etl_dag.py', "w") as file:
    file.write(modified_content)
    
with open(dags_sf_path / 'incremental_first_dag.py', "r") as file:
    content = file.read()
modified_content = content.replace('INPUT_SF', str(SF))
with open(dags_sf_path / 'incremental_first_dag.py', "w") as file:
    file.write(modified_content)

with open(dags_sf_path / 'incremental_second_dag.py', "r") as file:
    content = file.read()
modified_content = content.replace('INPUT_SF', str(SF))
with open(dags_sf_path / 'incremental_second_dag.py', "w") as file:
    file.write(modified_content)

In [5]:
output_file = dags_sf_path / 'staging_data_commands.sql'

# Generate SQL Statements
with open(output_file, "w") as file:
        sql = f"""COPY staging.batchdate FROM '{PATH}/Batch1/BatchDate.txt';\n
COPY staging.cashtransaction FROM '{PATH}/Batch1/CashTransaction.txt' delimiter '|';\n
COPY staging.dailymarket FROM '{PATH}/Batch1/DailyMarket.txt' delimiter '|';\n
COPY staging.date FROM '{PATH}/Batch1/Date.txt' delimiter '|';\n
COPY staging.holdinghistory FROM '{PATH}/Batch1/HoldingHistory.txt' delimiter '|';\n
COPY staging.hr FROM '{PATH}/Batch1/HR.csv' delimiter ',' CSV;\n
COPY staging.industry FROM '{PATH}/Batch1/Industry.txt' delimiter '|';\n
COPY staging.prospect FROM '{PATH}/Batch1/Prospect.csv' delimiter ',' CSV;\n
COPY staging.statustype FROM '{PATH}/Batch1/StatusType.txt' delimiter '|';\n
COPY staging.taxrate FROM '{PATH}/Batch1/TaxRate.txt' delimiter '|';\n
COPY staging.time FROM '{PATH}/Batch1/Time.txt' delimiter '|';\n
COPY staging.tradehistory FROM '{PATH}/Batch1/TradeHistory.txt' delimiter '|';\n
COPY staging.trade FROM '{PATH}/Batch1/Trade.txt' delimiter '|' null as '';\n
COPY staging.tradetype FROM '{PATH}/Batch1/TradeType.txt' delimiter '|';\n
COPY staging.watchhistory FROM '{PATH}/Batch1/WatchHistory.txt' delimiter '|';\n
COPY staging.audit FROM '{PATH}/Batch1_audit.csv' DELIMITER ',' HEADER CSV NULL AS '';\n       
"""

        file.write(sql)
        

In [17]:
output_file = dags_sf_path / 'staging_finwire_load1.sql'

# Generate SQL Statements
with open(output_file, "w") as file:
    for i in range(1967,2017):
        for j in range(4):
            sql = f"COPY staging.finwire FROM '{PATH}/Batch1/FINWIRE{i}Q{j+1}';\n"
            file.write(sql)
    sql = f"COPY staging.finwire FROM '{PATH}/Batch1/FINWIRE2017Q1';\n"
    file.write(sql)
    sql = f"COPY staging.finwire FROM '{PATH}/Batch1/FINWIRE2017Q2';\n"
    file.write(sql)
    sql = f"COPY staging.finwire FROM '{PATH}/Batch1/FINWIRE2017Q3';\n"
    file.write(sql)

In [6]:
output_file = dags_sf_path / "load_staging_customermgmt_db.sql"

# Generate SQL Statements
with open(output_file, "w") as file:
    sql = f"COPY staging.customermgmt FROM '{PATH}/Batch1/CustomerMgmt.csv' DELIMITER ',' HEADER CSV NULL AS '';\n"
    file.write(sql)

In [7]:
output_file = dags_sf_path / "incremental_update" / "load_staging_b3.sql"

# Generate SQL Statements
with open(output_file, "w") as file:
    sql = f"""COPY staging.batchdate FROM '{PATH}/Batch3/BatchDate.txt';
COPY staging.cashtransaction_b2 FROM '{PATH}/Batch3/CashTransaction.txt' delimiter '|';
COPY staging.dailymarket_b2 FROM '{PATH}/Batch3/DailyMarket.txt' delimiter '|';
COPY staging.holdinghistory_b2 FROM '{PATH}/Batch3/HoldingHistory.txt' delimiter '|';
COPY staging.prospect FROM '{PATH}/Batch3/Prospect.csv' delimiter ',' CSV;
COPY staging.watchhistory_b2 FROM '{PATH}/Batch3/WatchHistory.txt' delimiter '|';
COPY staging.trade_b2 FROM '{PATH}/Batch3/Trade.txt' delimiter '|' null as '';
COPY staging.customer FROM '{PATH}/Batch3/Customer.txt' delimiter '|' null as '';
COPY staging.account FROM '{PATH}/Batch3/Account.txt' delimiter '|' null as '';"""
    file.write(sql)

In [8]:
output_file = dags_sf_path / "incremental_update" / "load_staging.sql"

# Generate SQL Statements
with open(output_file, "w") as file:
    sql = f"""COPY staging.batchdate FROM '{PATH}/Batch2/BatchDate.txt';
COPY staging.cashtransaction_b2 FROM '{PATH}/Batch2/CashTransaction.txt' delimiter '|';
COPY staging.dailymarket_b2 FROM '{PATH}/Batch2/DailyMarket.txt' delimiter '|';
COPY staging.holdinghistory_b2 FROM '{PATH}/Batch2/HoldingHistory.txt' delimiter '|';
COPY staging.prospect FROM '{PATH}/Batch2/Prospect.csv' delimiter ',' CSV;
COPY staging.watchhistory_b2 FROM '{PATH}/Batch2/WatchHistory.txt' delimiter '|';
COPY staging.trade_b2 FROM '{PATH}/Batch2/Trade.txt' delimiter '|' null as '';
COPY staging.customer FROM '{PATH}/Batch2/Customer.txt' delimiter '|' null as '';
COPY staging.account FROM '{PATH}/Batch2/Account.txt' delimiter '|' null as '';"""
    file.write(sql)